# ! Exercise !

1. Open a Qiskit Runtime session with the `Estimator` context manager.
2. Make multiple requests to the `estimator` primitive within the session.
    * The parameter values of a request is based on the previous one.
    * The loop ends when a target is achieved.


### A Solution...

In the previous exercise, we computed the expectation values of the Pauli $X$ and $Y$ operators in the states $|1\rangle, |+\rangle$.

Here we will do the same but with a small twist. We will prepare a circuit in the $|1\rangle$ state and use Pauli $X$ operator if the expectation value is >0; Pauli $Y$ if the expectation value is <0. Since the expectation value should vanish, we set a target of 0.01 (not 0 because it's never _exactly_ 50/50).

In [1]:
# Authenticate with the server.
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")

In [2]:
# Prepare the circuit.
from qiskit import QuantumCircuit

qc1 = QuantumCircuit(1, 1)
qc1.x(0)
qc1.measure_all()

In [3]:
# Prepare the operators.
from qiskit.quantum_info import SparsePauliOp

x_op = SparsePauliOp.from_list([("X", 1)])
y_op = SparsePauliOp.from_list([("Y", 1)])

In [4]:
# Get the expectation values.
from qiskit_ibm_runtime import Estimator

with Estimator(
    circuits=[qc1],
    observables=[x_op, y_op],
    service=service, 
    options={ "backend": "ibmq_qasm_simulator" }
) as estimator:

    target = 0.01
    exp_val = estimator(circuits=[0], observables=[x_op]).values[0]
    print(f"Expectation value: {exp_val}")
    
    while abs(exp_val) > target:
        op = x_op if exp_val > 0 else y_op
        exp_val = estimator(circuits=[0], observables=[op]).values[0]
        print(f"Expectation value: {exp_val}")

Expectation value: 0.068359375
Expectation value: -0.060546875
Expectation value: -0.015625
Expectation value: 0.09375
Expectation value: 0.02734375
Expectation value: -0.037109375
Expectation value: -0.041015625
Expectation value: -0.0078125


In [5]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright